In [1]:
import pandas as pd 

In [7]:
# Import the dataset 
file = 'Data/data.csv'
dataframe = pd.read_csv(file, error_bad_lines=False, sep=";")

In [17]:
# Discrimination categories 
dataframe['Label'].value_counts()

none      11005
sexism     3110
racism     1932
none          2
Name: Label, dtype: int64

In [18]:
# Delete IDs 
dataframe.drop('ID', axis=1, inplace=True)

In [20]:
# First encoding 
dataframe['Label'].replace('none', 'not racist', inplace=True)
dataframe['Label'].replace('racism', 'racist', inplace=True)

In [21]:
# A copy of the original dataset 
df = dataframe.copy()

In [29]:
# First 20 tweets
pd.options.display.max_colwidth = 130  # to see long tweets 
df.head(20)

,Tweets,Label
0,@azzamalirhabi @JihadiA8 This video of the Peshmerga decimating ISIS is far more interesting. https://t.co/d36g1z12NP,not racist
1,Oh really? No more instant restaurants? THAT'S SHOCKING. #MKR #MKR2015,not racist
2,RT @Benfrancisallen: It hasn't been a good few weeks for #ISIS. A new front has opened up in #Sinjar and they're about to lose...,not racist
3,"RT @NoToFeminism: I donâ€™t need femisnsn because men carry heavy things that i cannot!!! like shopping, boxes, and a huge sen...",not racist
4,@MariachiMacabre 19% is not the vast majority,not racist
5,"@DianH4 @ExposeFalsehood And it is Muslims who were the first crusaders, attacking the Christian world for centuries before it...",racist
6,@truaemusic @mattybboi83 @Number10gov Capital Hill is a great example of how seldom the world attacks Islam given the daily pr...,racist
7,RT @fruitondabottom: #FeminismisEQUALITYwhen Men are actually listened to and part of the dialog. #HeForShe #WomenAgainstFemin...,sexism
8,"Gather round, kids. It's story time, brought to you by my good friend @jcmanous.",not racist
9,Did you hear that? That's the sound of a bunch of women in tech setting up filters to fwd emails from @ninaburleigh to /dev/nu...,not racist
